In [1]:
from __future__ import print_function

import sys

sys.path.append('src')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from model import Net


In [2]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         MaxPool2d-6           [-1, 16, 12, 12]               0
           Dropout-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 16, 14, 14]             272
              ReLU-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 12, 12]           2,320
             ReLU-12           [-1, 16, 12, 12]               0
      BatchNorm2d-13           [-1, 16, 12, 12]              32
        MaxPool2d-14             [-1, 1

/content/src/model.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [3]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),          # Rotate images randomly by ±30 degrees
                        transforms.ColorJitter(brightness=0.1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [4]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


loss=0.10725033283233643 batch_id=468: 100%|██████████| 469/469 [00:37<00:00, 12.53it/s]



Test set: Average loss: 0.0577, Accuracy: 9804/10000 (98.04%)

Epoch:  2


loss=0.04507721588015556 batch_id=468: 100%|██████████| 469/469 [00:32<00:00, 14.36it/s]



Test set: Average loss: 0.0350, Accuracy: 9893/10000 (98.93%)

Epoch:  3


...


Test set: Average loss: 0.0204, Accuracy: 9936/10000 (99.36%)

Epoch:  18


loss=0.007040448486804962 batch_id=468: 100%|██████████| 469/469 [00:32<00:00, 14.63it/s]



Test set: Average loss: 0.0209, Accuracy: 9938/10000 (99.38%)

Epoch:  19


loss=0.0126817487180233 batch_id=468: 100%|██████████| 469/469 [00:32<00:00, 14.33it/s]



Test set: Average loss: 0.0208, Accuracy: 9944/10000 (99.44%)



In [6]:
torch.save(model, 'mnist_best_model_99.44.pt')

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
! cp 'mnist_best_model_99.44.pt' '/content/drive/MyDrive'